In [5]:
import os
import pandas as pd

# get all full texts
list_of_files = []
length = 0
for index in range(1, 13):
    file_path = f"../data/pmc_patients/processed/full_texts_{index}.csv"
    if os.path.isfile(file_path):
        df = pd.read_csv(file_path)
        print(len(df))
        df.drop_duplicates(subset=["PMID"], keep="first", inplace=True)
        print(len(df))
        length += len(df)
        list_of_files.append(df)

print(length)

2945
2943
2945
2945
2945
2942
2945
2942
2944
2918
2944
2080
2944
2941
2944
2918
2944
2080
2944
1587
2944
1567
2944
1578
29441


In [6]:
# combine all full texts
df = pd.concat(list_of_files)
print(len(df))

# remove duplicates
df.drop_duplicates(subset=["PMID"], keep="first", inplace=True)
print(len(df))

# delete rows with empty full text
df = df.dropna(subset=["full_text"])
print(len(df))

# remove rows with failed downloads
df = df[~df.full_text.str.contains("Failed to download")]

print(len(df))
# update index
df = df.reset_index(drop=True)

29441
21500
21474
16889


In [7]:
df.to_csv("../data/pmc_patients/processed/full_texts_combined.csv")

## Remove patients with no full text available

In [8]:
print(len(df))
# get all full text paper ids
full_text_paper_ids = df["PMID"].tolist()

# get all relevant articles
list_of_articles = pd.read_csv("../data/pmc_patients/Summary_data/list_of_articles.csv")
print(len(list_of_articles))

# drop articles that are not in the list of full text papers
list_of_articles = list_of_articles[list_of_articles["article"].isin(full_text_paper_ids)]

print(len(list_of_articles))
list_of_articles.reset_index(drop=False, inplace=True)
list_of_articles.to_csv("../data/pmc_patients/Summary_data/list_of_articles_with_full_text.csv", index=False)

16889
128836
10178


In [ ]:
# import ast
# import pandas as pd

# patients = pd.read_csv("../data/pmc_patients/Summary_data/patient_relevant_articles_map.csv")
# list_of_papers = list_of_articles["article"].tolist()
# def flatten_list(input_list):
#     return [item for sublist in input_list for item in sublist]

# # drop articles that are not in the list of full text papers
# def filter_relevant_articles(relevant_articles):
#     articles = ast.literal_eval(relevant_articles)
#     return [article for article in articles if int(article) in list_of_papers]

# patients["relevant_articles"] = patients["relevant_articles"].apply(filter_relevant_articles)
# # drop row if relevant articles are empty
# patients = patients[patients["relevant_articles"].apply(len) > 0]
# patients.reset_index(drop=False, inplace=True)
# patients.to_csv("../data/pmc_patients/Summary_data/patient_relevant_articles_map_with_full_text.csv", index=False)

TypeError: list indices must be integers or slices, not str

In [10]:
patients

,index,patient,relevant_articles
0,0,This 60-year-old male was hospitalized due to ...,"[30427933, 12493078, 29208005]"
1,1,A 39-year-old man was hospitalized due to an i...,"[30427933, 12493078, 29208005]"
2,2,One week after a positive COVID-19 result this...,"[30427933, 12493078, 29208005]"
3,3,This 69-year-old male was admitted to the ICU ...,"[30427933, 12493078, 29208005]"
4,4,This 57-year-old male was admitted to the ICU ...,"[30427933, 12493078, 29208005]"
...,...,...,...
6007,9988,A 63-year-old woman with metastatic breast car...,[28814897]
6008,9989,"A 6 years old, neutered male Lhasa Apso was pr...","[30756087, 29057987, 29166400]"
6009,9990,"An 8 years old, neutered male mixed breed dog ...","[30756087, 29057987, 29166400]"
6010,9991,A 4 years old spayed female Doberman Pinscher ...,"[30756087, 29057987, 29166400]"


# Test paper access

In [11]:
import pandas as pd

full_text_papers = pd.read_csv("../data/pmc_patients/processed/full_texts_combined.csv")
full_text_papers_ids = pd.read_csv("../data/pmc_patients/Summary_data/list_of_articles_with_full_text.csv")

In [12]:
predictions = [89,120,181]

# input: list of paper indices
# map to paper ids
paper_ids = full_text_papers_ids[full_text_papers_ids["index"].isin(predictions)]
print(paper_ids)

papers = full_text_papers[full_text_papers["PMID"].isin(paper_ids["article"])]
papers

    index   article
5      89  22427859
8     120  16318699
13    181  19753282


,Unnamed: 0,PMID,full_text
289,289,16318699,Botulinum neurotoxins (BoNTs) are proteases th...
1064,1064,19753282,21\nCASE REPORT\nMJM 2009 12(1):21-24\nCopyrig...
2353,2353,22427859,Interferon-Gamma Release Assay Performance in\...


## create new maps for reduced dataset

In [13]:
import pandas as pd
import ast

short_description_df = pd.read_csv("../data/extracted_symptoms_on6k.csv")
short_description_df.head()

,Unnamed: 0,index,patient,relevant_articles,extracted_symptoms
0,0,0,This 60-year-old male was hospitalized due to ...,"['30427933', '12493078', '29208005']","['fever', 'cough']"
1,1,1,A 39-year-old man was hospitalized due to an i...,"['30427933', '12493078', '29208005']",['fever']
2,2,2,One week after a positive COVID-19 result this...,"['30427933', '12493078', '29208005']",[]
3,3,3,This 69-year-old male was admitted to the ICU ...,"['30427933', '12493078', '29208005']",[]
4,4,4,This 57-year-old male was admitted to the ICU ...,"['30427933', '12493078', '29208005']","['fever', 'cough']"


In [14]:
list_of_articles = short_description_df['relevant_articles'].tolist()
list_of_articles = [ast.literal_eval(string) for string in list_of_articles]
list_of_articles = flatten_list(list_of_articles)

set_of_articles = set(list_of_articles)
list_of_articles = list(set_of_articles)
dict_of_articles = {article: index for index, article in enumerate(list_of_articles)}
df = pd.DataFrame(list_of_articles, columns=['article'])
df.to_csv('list_of_articles.csv', index=True)
df

,article
0,20507748
1,18787367
2,26918139
3,29425193
4,25276077
...,...
10173,23139750
10174,21629722
10175,29988290
10176,28058067


In [16]:
len(dict_of_articles)

10178

In [26]:


patient_relevant_articles_map = pd.read_csv("../data/extracted_symptoms_on6k.csv")
patient_relevant_articles_map['relevant_articles'] = patient_relevant_articles_map['relevant_articles'].apply(ast.literal_eval)

for index, row in patient_relevant_articles_map.iterrows():
    value = row['relevant_articles']
    #value = ast.literal_eval(value)
    new_list = []
    for paper in value:
        new_list.append(dict_of_articles[paper])
    patient_relevant_articles_map.at[index, 'relevant_articles'] = new_list
patient_relevant_articles_map.to_csv('patient_relevant_articles_ids_map_with_full_text.csv', index=False)
len(patient_relevant_articles_map)
# Filtern der Zeilen mit leeren 'relevant_articles'
empty_articles = patient_relevant_articles_map[patient_relevant_articles_map['relevant_articles'].apply(lambda x: len(x) == 0)]

# Anzahl und Inhalt der leeren Listen anzeigen
empty_count = len(empty_articles)
print(f"Anzahl der leeren 'relevant_articles': {empty_count}")
print("Inhalt der leeren 'relevant_articles':")
print(empty_articles)

patient_relevant_articles_map.head()

Anzahl der leeren 'relevant_articles': 0
Inhalt der leeren 'relevant_articles':
Empty DataFrame
Columns: [Unnamed: 0, index, patient, relevant_articles, extracted_symptoms]
Index: []


,Unnamed: 0,index,patient,relevant_articles,extracted_symptoms
0,0,0,This 60-year-old male was hospitalized due to ...,"[8440, 3117, 8425]","['fever', 'cough']"
1,1,1,A 39-year-old man was hospitalized due to an i...,"[8440, 3117, 8425]",['fever']
2,2,2,One week after a positive COVID-19 result this...,"[8440, 3117, 8425]",[]
3,3,3,This 69-year-old male was admitted to the ICU ...,"[8440, 3117, 8425]",[]
4,4,4,This 57-year-old male was admitted to the ICU ...,"[8440, 3117, 8425]","['fever', 'cough']"


In [27]:
# drow rows where relevant articles are empty
patient_relevant_articles_map = patient_relevant_articles_map[patient_relevant_articles_map["extracted_symptoms"].apply(len) > 2]
patient_relevant_articles_map.reset_index(drop=True, inplace=True)
patient_relevant_articles_map.to_csv('patient_relevant_articles_ids_map_full_text_short_description.csv', index=False)
patient_relevant_articles_map

,Unnamed: 0,index,patient,relevant_articles,extracted_symptoms
0,0,0,This 60-year-old male was hospitalized due to ...,"[7926, 5730, 10140]","['fever', 'cough']"
1,1,1,A 39-year-old man was hospitalized due to an i...,"[7926, 5730, 10140]",['fever']
2,4,4,This 57-year-old male was admitted to the ICU ...,"[7926, 5730, 10140]","['fever', 'cough']"
3,5,5,This 52-year-old male tested COVID-19 positive...,"[7926, 5730, 10140]",['fever']
4,7,7,This 33-year-old female patient had typical CO...,"[7926, 5730, 10140]","['fever', 'headache', 'delirium', 'delirium']"
...,...,...,...,...,...
3651,6004,9982,A 21-year-old female presented with fatigue an...,[6378],"['fatigue', 'palpitations', 'palpitations', 'f..."
3652,6005,9983,A 39-year-old female presented with fatigue an...,[6378],"['fatigue', 'jaundice']"
3653,6006,9984,A 38-year-old female presented with marked jau...,[6378],"['jaundice', 'tremors', 'palpitations']"
3654,6008,9989,"A 6 years old, neutered male Lhasa Apso was pr...","[10141, 7524, 365]",['weakness']


In [8]:
import spacy
import pandas as pd
import scispacy
from scispacy.linking import EntityLinker
from spacy import displacy
import os
from tqdm import tqdm

#from src.utils.constants import symptom_list

# Load the SciSpaCy model for medical text
nlp = spacy.load("en_core_sci_sm")

# # Example text (Symptoms in medical records)
#text = "The patient shows signs of fever, continuous coughing, and shortness of breath."

# # List of symptoms to match
# symptom_list = [
#     "fever", "cough", "shortness of breath", "headache", "fatigue", "nausea", "vomiting", "sore throat", "chills", "dizziness", "chest pain", "muscle aches", "loss of appetite", "sweating", "runny nose", "sneezing", "congestion", "body aches", "joint pain", "sore muscles", "diarrhea", "abdominal pain", "rash", "skin redness", "swelling", "coughing up blood", "wheezing", "blurry vision", "insomnia", "irritability", "confusion", "numbness", "tingling", "hearing loss", "difficulty swallowing", "hoarseness", "fatigue", "hypertension", "bradycardia", "tachycardia", "hypotension", "hyperglycemia", "hypoglycemia", "seizures", "syncope", "palpitations", "lightheadedness", "chest tightness", "wheezing", "flank pain", "hematuria", "oliguria", "anuria", "urinary retention", "incontinence", "elevated heart rate", "rapid breathing", "persistent cough", "night sweats", "bleeding gums", "jaundice", "hepatomegaly", "splenomegaly", "back pain", "visual disturbances", "weight loss", "fever spikes", "delirium", "restlessness", "tingling in hands", "weakness", "difficulty breathing", "swollen lymph nodes", "nausea with vomiting", "constant hunger", "polydipsia", "hearing ringing", "poor circulation", "cold hands and feet", "confusion", "visual floaters", "sensitivity to light", "difficulty walking", "fatigue after exertion", "unexplained bruising", "bone pain", "skin ulcer", "fainting", "sore throat with swollen lymph nodes", "sensitivity to cold", "pins and needles", "thickened skin", "dark-colored urine", "increased thirst", "dry mouth", "red eyes", "bloody stools", "chronic cough", "persistent fatigue", "yellowing of skin", "swollen ankles", "numbness in feet", "chronic fatigue", "palpitations with dizziness", "swollen abdomen", "tremors", "muscle weakness", "spitting blood", "persistent fever", "low blood pressure", "wheezing with coughing", "irregular heartbeat", "loss of consciousness", "abnormal breathing patterns", "confusion and disorientation", "blurred vision", "sudden weight loss", "unexplained fever", "fatigue after meals", "stomach cramps", "digestive problems", "gastrointestinal bleeding", "slow heart rate", "rash with itching", "ringing in ears", "Shortness of breath", "Wheezing", "Persistent cough", "Coughing up blood", "Chest tightness", "Chest pain", "Palpitations", "Irregular heartbeat", "Dizziness with exertion", "Difficulty breathing", "Rapid breathing", "Decreased lung sounds", "Pulmonary edema", "Stridor", "Rales", "Hypoxemia", "Hyperventilation", "Hypoventilation", "Labored breathing", "Frequent shallow breathing", "Sighing", "Nasal flaring", "Cyanosis", "Bluish lips", "Bluish fingertips", "Hypercapnia", "Crackling in chest", "Pain with deep breathing", "Pressure on chest", "Increased respiratory rate", "Swollen ankles", "Cold extremities", "Claudication", "Lower extremity edema", "Leg swelling", "Peripheral edema", "Elevated blood pressure", "Hypotension", "Orthopnea", "Paroxysmal nocturnal dyspnea", "Jugular vein distension", "Tachycardia", "Bradycardia", "Chest discomfort", "Elevated heart rate", "Weak pulse", "Bounding pulse", "Heart murmur", "Angina", "Myocardial infarction", "Aortic stenosis", "Coronary artery disease", "Fainting spells", "Lightheadedness", "Pulse irregularities", "Swollen veins", "Cardiomegaly", "Pulmonary hypertension", "Arrhythmia", "Heart palpitations", "Cardiac arrest", "Atrial fibrillation", "Systolic hypertension", "Diastolic hypertension", "Exertional dyspnea", "S3 heart sound", "S4 heart sound", "Rapid heart rate with exercise", "Decreased exercise tolerance", "Sudden decrease in blood pressure", "Tachypnea", "Hypotension with dizziness", "Elevated JVP", "Excessive perspiration", "Cold sweats", "Heart failure", "Peripheral vascular disease", "Pulmonary embolism", "Tachycardic rhythm", "Stiff arteries", "Pulmonary consolidation", "Sputum production", "Pleural effusion", "Acid reflux with chest pain", "Intermittent claudication", "Pitting edema in legs", "Persistent blood pressure fluctuations", "Exercise-induced wheezing", "Stiffness in the chest", "Pain radiating to the jaw", "Pain radiating to the left arm", "Sensation of pressure in chest", "Dizziness upon standing", "Fatigue with minimal exertion", "Elevated creatine kinase levels", "Abnormal ECG findings", "Dyspnea on exertion", "Dyspnea at rest", "Discomfort in upper chest", "Unexplained rapid heartbeat", "Unexplained shortness of breath", "Tightness in chest", "Increased heart rate after activity", "Decreased heart rate with activity", "Unusual heartbeat", "Frequent heart skips", "Shortness of breath at rest", "Pain with deep inspiration", "Increased heart rate at night", "Sudden drop in blood pressure", "Sudden increase in blood pressure", "Atypical chest pain", "Pale or ashen skin tone", "Sensation of chest fluttering", "Difficulty breathing after physical activity", "Increased heart rate with stress", "Difficulty catching breath after exertion", "Uncontrolled coughing", "Elevated systolic blood pressure", "Sweaty palms", "Increased heart rate at night", "Cold hands and feet with exercise", "Pulmonary crackles", "Rapid pulse during rest", "Heart pounding in chest", "Cardiovascular instability", "Pain on exertion", "Sudden chest tightness", "Pain when lying down", "Tingling sensation in chest", "Chronic cough with exertion", "Abnormal heart rhythms with exercise", "Heavy feeling in chest", "Inability to catch breath", "Sensation of heart skipping beats", "Swollen eyelids", "Muscle stiffness", "Weak legs", "Tingling toes", "Restless legs", "Skin blistering", "Dry cough", "Slow speech", "Burning mouth", "Difficulty raising arms", "Sensory overload", "Hair breakage", "Sleepwalking", "Sudden aggression", "Arm weakness", "Unexplained rash", "Excess mucus in throat", "Stomach noises", "Ear pain on swallowing", "Swollen tongue", "Face pain", "Muscle jerking", "Uncontrolled blinking", "Gurgling stomach", "Cough with phlegm", "Lower abdominal pain", "White spots on tonsils", "Scaly patches", "Difficulty closing eyes", "Poor hand coordination", "Nausea without vomiting", "Stiff upper lip", "Hoarseness without sore throat", "Excessive blinking", "Unexplained jaw pain", "Weakness in hands", "Tingling lips", "Dark patches on skin", "Shivering without fever", "Cold sweats", "Lump in breast", "Pain in upper arm", "Bloodshot sclera", "Frequent burping", "Muscle tightness", "Stomach bloating after meals", "Dry itchy scalp", "Eye strain", "Slow digestion", "Frequent ear infections", "Drooping eyelid", "Unusual sweating pattern", "Wobbly legs", "Delayed speech", "Irregular bowel movements", "Pain in shins", "Chronic sinus infections", "Puffy hands", "Burning legs", "Unexplained choking sensation", "Burning in throat", "Recurrent mouth ulcers", "Inability to relax", "Stomach twitching", "Twitching fingers", "Increased heart palpitations", "Constant buzzing in ears", "Restless arms", "Swollen upper lip", "Sudden weakness in arms", "Red, irritated skin", "Muscle spasms in back", "Facial twitching", "Twitching under eye", "Sharp chest pains", "Frequent hiccups", "Sharp shoulder pain", "Sudden urge to urinate", "White coating on tongue", "Loss of grip strength", "Short-term memory issues", "Loss of bladder sensation", "Pale gums", "Frequent nose rubbing", "Recurrent swollen feet", "Pounding in ears", "Skin sagging", "Tearing eyes", "Bloating with cramps", "Blue lips with cold", "Pulsating pain in head", "Yellow tongue coating", "Pain in lower ribcage", "Hot flashes at night", "Tight chest with exertion", "Swollen jaw", "Tired eyes", "Slow pulse", "Difficulty waking up", "Dry nostrils", "Nightmares with sweating", "Loss of interest in hobbies", "Persistent body odor", "Sudden head pressure", "Jaw locking", "Burning sensation in stomach", "Frequent bruising on legs", "Sudden excessive thirst", "Sensitivity to noise", "Recurrent fevers", "Puffy ankles", "Discoloration of fingers", "Poor skin elasticity", "Abnormal heart sounds", "Frequent congestion", "Tingling ears", "Stiff shoulders", "Hot hands and feet", "Fever", "Chills", "Fatigue", "Weakness", "Weight loss", "Weight gain", "Loss of appetite"
# ]

# Using set instead of list for faster search
# symptom_set = set(map(str.lower, symptom_list))
def extract_symptoms(text):
    symptoms = [str(word).lower() for word in nlp(text).ents]
    symptoms = set(symptoms)
    symptoms = sorted(symptoms)
    return symptoms

# # Load the dataset from CSV
df = pd.read_csv('/home/dominik/Documents/Informatik/1_Semester/medLLM/data/pmc_patients/PMC-Patients-oa-9995.csv')

tqdm.pandas(desc="Extracting symptoms")

# # Assuming 'patient' column contains the medical records. If not, change this to the correct column name.
df['extracted_symptoms'] = df['patient'].head(20).progress_apply(lambda x: extract_symptoms(x))

# # Save the results to a new CSV file
df.to_csv('extracted_symptoms_20.csv', index=False)

# # Display the extracted symptoms
print(df[['patient_uid', 'extracted_symptoms']])


# class SymptomExtractor:
#     def __init__(self):
#         # Load the SciSpaCy model for medical text
#         self.nlp = spacy.load("en_core_sci_sm")
#         # List of symptoms to match

#         # Convert the list of symptoms to a set for fast lookup
#         self.symptom_set = {symptom.lower() for symptom in symptom_list}
    
#     def extract_symptoms(self, text):
#         # Extract symptoms from the provided text
#         return [
#             word.text for word in self.nlp(text).ents #if word.text.lower() in self.symptom_set
#         ]

    # Example usage:

    # Create an instance of SymptomExtractor with your symptom list
    #symptom_extractor = SymptomExtractor(symptom_list)

    # Example input text for the extraction
    #text = "The patient shows signs of fever, continuous coughing, and shortness of breath."

    # Extract symptoms from the text
    #extracted_symptoms = symptom_extractor.extract_symptoms(text)
    #print("Extracted Symptoms:", extracted_symptoms)

Extracting symptoms: 100%|██████████| 20/20 [00:02<00:00,  7.20it/s]


     patient_uid                                 extracted_symptoms
0      7665777-1  [acute ward, air distribution, ards, breaks, b...
1      7665777-2  [aggravated, alleviate, anxiety, anxious, bare...
2      7665777-3  [admitted, breathing frequency, convalescence,...
3      7665777-4  [active participation, admitted, ards, asynchr...
4      7665777-5  [abdomen, admitted, airway, alveolar collapse,...
...          ...                                                ...
9988   6624466-1                                                NaN
9989   6624786-1                                                NaN
9990   6624786-2                                                NaN
9991   6624786-3                                                NaN
9992   6624786-4                                                NaN

[9993 rows x 2 columns]


In [9]:
df


,index,patient_uid,PMID,file_path,title,patient,age,gender,relevant_articles,similar_patients,extracted_symptoms
0,0,7665777-1,33492400,comm/PMC007xxxxxx/PMC7665777.xml,Early Physical Therapist Interventions for Pat...,This 60-year-old male was hospitalized due to ...,"[[60.0, 'year']]",M,"{'32320506': 1, '32293716': 1, '23219649': 1, ...","{'7665777-2': 2, '7665777-3': 2, '7665777-4': ...","[acute ward, air distribution, ards, breaks, b..."
1,1,7665777-2,33492400,comm/PMC007xxxxxx/PMC7665777.xml,Early Physical Therapist Interventions for Pat...,A 39-year-old man was hospitalized due to an i...,"[[39.0, 'year']]",M,"{'32320506': 1, '32293716': 1, '23219649': 1, ...","{'7665777-1': 2, '7665777-3': 2, '7665777-4': ...","[aggravated, alleviate, anxiety, anxious, bare..."
2,2,7665777-3,33492400,comm/PMC007xxxxxx/PMC7665777.xml,Early Physical Therapist Interventions for Pat...,One week after a positive COVID-19 result this...,"[[57.0, 'year']]",M,"{'32320506': 1, '32293716': 1, '23219649': 1, ...","{'7665777-1': 2, '7665777-2': 2, '7665777-4': ...","[admitted, breathing frequency, convalescence,..."
3,3,7665777-4,33492400,comm/PMC007xxxxxx/PMC7665777.xml,Early Physical Therapist Interventions for Pat...,This 69-year-old male was admitted to the ICU ...,"[[69.0, 'year']]",M,"{'32320506': 1, '32293716': 1, '23219649': 1, ...","{'7665777-1': 2, '7665777-2': 2, '7665777-3': ...","[active participation, admitted, ards, asynchr..."
4,4,7665777-5,33492400,comm/PMC007xxxxxx/PMC7665777.xml,Early Physical Therapist Interventions for Pat...,This 57-year-old male was admitted to the ICU ...,"[[57.0, 'year']]",M,"{'32320506': 1, '32293716': 1, '23219649': 1, ...","{'7665777-1': 2, '7665777-2': 2, '7665777-3': ...","[abdomen, admitted, airway, alveolar collapse,..."
...,...,...,...,...,...,...,...,...,...,...,...
9988,9995,6624466-1,31245980,comm/PMC006xxxxxx/PMC6624466.xml,Breast Carcinoma Metastasis to the Medial Rect...,A 63-year-old woman with metastatic breast car...,"[[63.0, 'year']]",F,"{'15756132': 1, '26178006': 1, '28814897': 1, ...","{'3159321-1': 1, '8385588-1': 1}",NaN
9989,9996,6624786-1,31334255,comm/PMC006xxxxxx/PMC6624786.xml,"Canine Snake-Eye Myelopathy: Clinical, Magneti...","A 6 years old, neutered male Lhasa Apso was pr...","[[6.0, 'year']]",M,"{'21889166': 1, '30756087': 1, '27616317': 1, ...","{'6624786-2': 2, '6624786-3': 2, '6624786-4': 2}",NaN
9990,9997,6624786-2,31334255,comm/PMC006xxxxxx/PMC6624786.xml,"Canine Snake-Eye Myelopathy: Clinical, Magneti...","An 8 years old, neutered male mixed breed dog ...","[[8.0, 'year']]",M,"{'21889166': 1, '30756087': 1, '27616317': 1, ...","{'6624786-1': 2, '6624786-3': 2, '6624786-4': 2}",NaN
9991,9998,6624786-3,31334255,comm/PMC006xxxxxx/PMC6624786.xml,"Canine Snake-Eye Myelopathy: Clinical, Magneti...",A 4 years old spayed female Doberman Pinscher ...,"[[4.0, 'year']]",F,"{'21889166': 1, '30756087': 1, '27616317': 1, ...","{'6624786-1': 2, '6624786-2': 2, '6624786-4': 2}",NaN


In [ ]:
['anxiety', 'covid-19', 'cpax', 'day', 'days', 'delirium', 'die', 'dry cough', 'dysphagia', 'dyspnea', 'exceptional', 'extubation', 'female', 'functional activities', 'headache', 'high fever', 'hospital visits', 'husband', 'icu admission', 'icu-acquired weakness', 'intubated', 'limiting', 'mobilized', 'mrc-ss', 'onwards', 'pandemic measures', 'participate', 'passively', 'patient', 'peripheral acute hospital', 'permission', 'physical therapy', 'postextubation', 'prevention', 'prone', 'proned', 'rapidly', 'rehabilitation', 'respiratory deterioration', 'secondary damage', 'security', 'severe', 'side-edge position', 'situation', 'swiss government', 'symptoms', 'threatened', 'unstable', 'visit', 'week']

In [10]:
df.head()

,index,patient_uid,PMID,file_path,title,patient,age,gender,relevant_articles,similar_patients,extracted_symptoms
0,0,7665777-1,33492400,comm/PMC007xxxxxx/PMC7665777.xml,Early Physical Therapist Interventions for Pat...,This 60-year-old male was hospitalized due to ...,"[[60.0, 'year']]",M,"{'32320506': 1, '32293716': 1, '23219649': 1, ...","{'7665777-2': 2, '7665777-3': 2, '7665777-4': ...","[acute ward, air distribution, ards, breaks, b..."
1,1,7665777-2,33492400,comm/PMC007xxxxxx/PMC7665777.xml,Early Physical Therapist Interventions for Pat...,A 39-year-old man was hospitalized due to an i...,"[[39.0, 'year']]",M,"{'32320506': 1, '32293716': 1, '23219649': 1, ...","{'7665777-1': 2, '7665777-3': 2, '7665777-4': ...","[aggravated, alleviate, anxiety, anxious, bare..."
2,2,7665777-3,33492400,comm/PMC007xxxxxx/PMC7665777.xml,Early Physical Therapist Interventions for Pat...,One week after a positive COVID-19 result this...,"[[57.0, 'year']]",M,"{'32320506': 1, '32293716': 1, '23219649': 1, ...","{'7665777-1': 2, '7665777-2': 2, '7665777-4': ...","[admitted, breathing frequency, convalescence,..."
3,3,7665777-4,33492400,comm/PMC007xxxxxx/PMC7665777.xml,Early Physical Therapist Interventions for Pat...,This 69-year-old male was admitted to the ICU ...,"[[69.0, 'year']]",M,"{'32320506': 1, '32293716': 1, '23219649': 1, ...","{'7665777-1': 2, '7665777-2': 2, '7665777-3': ...","[active participation, admitted, ards, asynchr..."
4,4,7665777-5,33492400,comm/PMC007xxxxxx/PMC7665777.xml,Early Physical Therapist Interventions for Pat...,This 57-year-old male was admitted to the ICU ...,"[[57.0, 'year']]",M,"{'32320506': 1, '32293716': 1, '23219649': 1, ...","{'7665777-1': 2, '7665777-2': 2, '7665777-3': ...","[abdomen, admitted, airway, alveolar collapse,..."


In [21]:
from collections import Counter
import ast
# Beispiel DataFrame
# Zähle die Häufigkeit der PMIDs
pmid_counts = df['PMID'].value_counts()

#df['relevant_articles'] = df['relevant_articles'].apply(ast.literal_eval)
# Zählen der Häufigkeit von relevant_article IDs
relevant_article_flat_list = []
for articles in df['relevant_articles']:
    relevant_article_flat_list.extend(articles.keys())

relevant_article_counts = Counter(relevant_article_flat_list)
sorted_relevant_articles = dict(sorted(relevant_article_counts.items(), key=lambda item: item[1], reverse=True))

print("Häufigkeit der PMIDs:")
print(pmid_counts)

print("\nHäufigkeit der relevant article IDs:")
print(relevant_article_counts)

Häufigkeit der PMIDs:
PMID
31293970    13
33492400    11
30147646    11
31278606    11
30854332    10
            ..
30402322     1
30326861     1
30318016     1
30402312     1
30326852     1
Name: count, Length: 8425, dtype: int64

Häufigkeit der relevant article IDs:
Counter({'27613565': 135, '30342279': 110, '25741868': 79, '27535533': 44, '27770639': 41, '26027431': 39, '7249508': 36, '26980727': 32, '20525992': 32, '25891173': 30, '29313949': 30, '16937360': 30, '20644199': 29, '27157931': 28, '27069254': 25, '21325604': 25, '25795410': 25, '29320654': 24, '20354512': 24, '24590637': 23, '19097774': 22, '26373316': 21, '26446948': 21, '27646942': 21, '27307501': 21, '29689030': 21, '22316447': 20, '27282937': 20, '28973656': 20, '28666240': 20, '27718847': 20, '28405474': 20, '28076863': 19, '28055103': 19, '27806233': 19, '20080505': 19, '28495807': 18, '20180029': 18, '20601685': 18, '19451168': 18, '24482447': 18, '29162153': 18, '29125905': 18, '28821685': 18, '29600292': 17, 

In [20]:
relevant_articles_more_than_once = sum(1 for count in relevant_article_counts.values() if count > 1)
relevant_articles_more_than_once

27868

In [24]:
relevant_article_flat_list = []
for articles in df['relevant_articles']:
    relevant_article_flat_list.extend(articles.keys())

relevant_article_counts = Counter(relevant_article_flat_list)

# Ermittlung der IDs, die mindestens zweimal vorkommen
ids_to_keep = {article_id for article_id, count in relevant_article_counts.items() if count >= 2}

# Filtern der relevant_articles
df['filtered_relevant_articles'] = df['relevant_articles'].apply(
    lambda articles: {article_id: count for article_id, count in articles.items() if article_id in ids_to_keep}
)

In [25]:
df

,index,patient_uid,PMID,file_path,title,patient,age,gender,relevant_articles,similar_patients,extracted_symptoms,filtered_relevant_articles
0,0,7665777-1,33492400,comm/PMC007xxxxxx/PMC7665777.xml,Early Physical Therapist Interventions for Pat...,This 60-year-old male was hospitalized due to ...,"[[60.0, 'year']]",M,"{'32320506': 1, '32293716': 1, '23219649': 1, ...","{'7665777-2': 2, '7665777-3': 2, '7665777-4': ...","[acute ward, air distribution, ards, breaks, b...","{'32320506': 1, '32293716': 1, '23219649': 1, ..."
1,1,7665777-2,33492400,comm/PMC007xxxxxx/PMC7665777.xml,Early Physical Therapist Interventions for Pat...,A 39-year-old man was hospitalized due to an i...,"[[39.0, 'year']]",M,"{'32320506': 1, '32293716': 1, '23219649': 1, ...","{'7665777-1': 2, '7665777-3': 2, '7665777-4': ...","[aggravated, alleviate, anxiety, anxious, bare...","{'32320506': 1, '32293716': 1, '23219649': 1, ..."
2,2,7665777-3,33492400,comm/PMC007xxxxxx/PMC7665777.xml,Early Physical Therapist Interventions for Pat...,One week after a positive COVID-19 result this...,"[[57.0, 'year']]",M,"{'32320506': 1, '32293716': 1, '23219649': 1, ...","{'7665777-1': 2, '7665777-2': 2, '7665777-4': ...","[admitted, breathing frequency, convalescence,...","{'32320506': 1, '32293716': 1, '23219649': 1, ..."
3,3,7665777-4,33492400,comm/PMC007xxxxxx/PMC7665777.xml,Early Physical Therapist Interventions for Pat...,This 69-year-old male was admitted to the ICU ...,"[[69.0, 'year']]",M,"{'32320506': 1, '32293716': 1, '23219649': 1, ...","{'7665777-1': 2, '7665777-2': 2, '7665777-3': ...","[active participation, admitted, ards, asynchr...","{'32320506': 1, '32293716': 1, '23219649': 1, ..."
4,4,7665777-5,33492400,comm/PMC007xxxxxx/PMC7665777.xml,Early Physical Therapist Interventions for Pat...,This 57-year-old male was admitted to the ICU ...,"[[57.0, 'year']]",M,"{'32320506': 1, '32293716': 1, '23219649': 1, ...","{'7665777-1': 2, '7665777-2': 2, '7665777-3': ...","[abdomen, admitted, airway, alveolar collapse,...","{'32320506': 1, '32293716': 1, '23219649': 1, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...
9988,9995,6624466-1,31245980,comm/PMC006xxxxxx/PMC6624466.xml,Breast Carcinoma Metastasis to the Medial Rect...,A 63-year-old woman with metastatic breast car...,"[[63.0, 'year']]",F,"{'15756132': 1, '26178006': 1, '28814897': 1, ...","{'3159321-1': 1, '8385588-1': 1}",NaN,"{'15756132': 1, '26178006': 1, '34457303': 2, ..."
9989,9996,6624786-1,31334255,comm/PMC006xxxxxx/PMC6624786.xml,"Canine Snake-Eye Myelopathy: Clinical, Magneti...","A 6 years old, neutered male Lhasa Apso was pr...","[[6.0, 'year']]",M,"{'21889166': 1, '30756087': 1, '27616317': 1, ...","{'6624786-2': 2, '6624786-3': 2, '6624786-4': 2}",NaN,"{'21889166': 1, '30756087': 1, '27616317': 1, ..."
9990,9997,6624786-2,31334255,comm/PMC006xxxxxx/PMC6624786.xml,"Canine Snake-Eye Myelopathy: Clinical, Magneti...","An 8 years old, neutered male mixed breed dog ...","[[8.0, 'year']]",M,"{'21889166': 1, '30756087': 1, '27616317': 1, ...","{'6624786-1': 2, '6624786-3': 2, '6624786-4': 2}",NaN,"{'21889166': 1, '30756087': 1, '27616317': 1, ..."
9991,9998,6624786-3,31334255,comm/PMC006xxxxxx/PMC6624786.xml,"Canine Snake-Eye Myelopathy: Clinical, Magneti...",A 4 years old spayed female Doberman Pinscher ...,"[[4.0, 'year']]",F,"{'21889166': 1, '30756087': 1, '27616317': 1, ...","{'6624786-1': 2, '6624786-2': 2, '6624786-4': 2}",NaN,"{'21889166': 1, '30756087': 1, '27616317': 1, ..."


In [28]:
df.to_csv("extended_extracted_symptoms.csv")

In [27]:
df['extended_extracted_symptoms'] = df['patient'].progress_apply(extract_symptoms)


Extracting symptoms: 100%|██████████| 9993/9993 [10:39<00:00, 15.62it/s]


In [80]:
df = pd.read_csv("extended_extracted_symptoms.csv")

In [73]:
df.head()

,Unnamed: 0,index,patient_uid,PMID,file_path,title,patient,age,gender,relevant_articles,similar_patients,extracted_symptoms,filtered_relevant_articles,extended_extracted_symptoms
0,0,0,7665777-1,33492400,comm/PMC007xxxxxx/PMC7665777.xml,Early Physical Therapist Interventions for Pat...,This 60-year-old male was hospitalized due to ...,"[[60.0, 'year']]",M,"{'32320506': 1, '32293716': 1, '23219649': 1, ...","{'7665777-2': 2, '7665777-3': 2, '7665777-4': ...","['acute ward', 'air distribution', 'ards', 'br...","{'32320506': 1, '32293716': 1, '23219649': 1, ...","['acute ward', 'air distribution', 'ards', 'br..."
1,1,1,7665777-2,33492400,comm/PMC007xxxxxx/PMC7665777.xml,Early Physical Therapist Interventions for Pat...,A 39-year-old man was hospitalized due to an i...,"[[39.0, 'year']]",M,"{'32320506': 1, '32293716': 1, '23219649': 1, ...","{'7665777-1': 2, '7665777-3': 2, '7665777-4': ...","['aggravated', 'alleviate', 'anxiety', 'anxiou...","{'32320506': 1, '32293716': 1, '23219649': 1, ...","['aggravated', 'alleviate', 'anxiety', 'anxiou..."
2,2,2,7665777-3,33492400,comm/PMC007xxxxxx/PMC7665777.xml,Early Physical Therapist Interventions for Pat...,One week after a positive COVID-19 result this...,"[[57.0, 'year']]",M,"{'32320506': 1, '32293716': 1, '23219649': 1, ...","{'7665777-1': 2, '7665777-2': 2, '7665777-4': ...","['admitted', 'breathing frequency', 'convalesc...","{'32320506': 1, '32293716': 1, '23219649': 1, ...","['admitted', 'breathing frequency', 'convalesc..."
3,3,3,7665777-4,33492400,comm/PMC007xxxxxx/PMC7665777.xml,Early Physical Therapist Interventions for Pat...,This 69-year-old male was admitted to the ICU ...,"[[69.0, 'year']]",M,"{'32320506': 1, '32293716': 1, '23219649': 1, ...","{'7665777-1': 2, '7665777-2': 2, '7665777-3': ...","['active participation', 'admitted', 'ards', '...","{'32320506': 1, '32293716': 1, '23219649': 1, ...","['active participation', 'admitted', 'ards', '..."
4,4,4,7665777-5,33492400,comm/PMC007xxxxxx/PMC7665777.xml,Early Physical Therapist Interventions for Pat...,This 57-year-old male was admitted to the ICU ...,"[[57.0, 'year']]",M,"{'32320506': 1, '32293716': 1, '23219649': 1, ...","{'7665777-1': 2, '7665777-2': 2, '7665777-3': ...","['abdomen', 'admitted', 'airway', 'alveolar co...","{'32320506': 1, '32293716': 1, '23219649': 1, ...","['abdomen', 'admitted', 'airway', 'alveolar co..."


In [81]:
new_df = df[['PMID', 'filtered_relevant_articles', 'extended_extracted_symptoms']].copy()
new_df['filtered_relevant_articles'] = new_df['filtered_relevant_articles'].apply(ast.literal_eval)

new_df['filtered_relevant_articles'] = new_df['filtered_relevant_articles'].apply(lambda articles: list(articles.keys()))

# new_df['filtered_relevant_articles'] = new_df['filtered_relevant_articles'].apply(
#     lambda articles: sorted(articles) if articles else None
# )

In [82]:
new_df.head()

,PMID,filtered_relevant_articles,extended_extracted_symptoms
0,33492400,"[32320506, 32293716, 23219649, 30339549, 17470...","['acute ward', 'air distribution', 'ards', 'br..."
1,33492400,"[32320506, 32293716, 23219649, 30339549, 17470...","['aggravated', 'alleviate', 'anxiety', 'anxiou..."
2,33492400,"[32320506, 32293716, 23219649, 30339549, 17470...","['admitted', 'breathing frequency', 'convalesc..."
3,33492400,"[32320506, 32293716, 23219649, 30339549, 17470...","['active participation', 'admitted', 'ards', '..."
4,33492400,"[32320506, 32293716, 23219649, 30339549, 17470...","['abdomen', 'admitted', 'airway', 'alveolar co..."


In [65]:
def flatten_list(input_list):
    return [item for sublist in input_list for item in sublist]


In [83]:
list_of_articles = new_df['filtered_relevant_articles'].tolist()

# list_of_articles = [ast.literal_eval(string) for string in list_of_articles]
list_of_articles = flatten_list(list_of_articles)
set_of_articles = set(list_of_articles)
list_of_articles = list(set_of_articles)
dict_of_articles = {article: index for index, article in enumerate(list_of_articles)}
articles_mapping_df = pd.DataFrame(list_of_articles, columns=['article'])
articles_mapping_df
articles_mapping_df.to_csv('extended_list_of_articles.csv', index=True)

In [77]:
articles_mapping_df.head()

,article
0,12216782
1,2194726
2,18492102
3,34590016
4,23036289


In [86]:
dict_of_articles

{'12216782': 0,
 '2194726': 1,
 '18492102': 2,
 '34590016': 3,
 '23036289': 4,
 '30418525': 5,
 '16130612': 6,
 '34720980': 7,
 '11745279': 8,
 '15543910': 9,
 '27114917': 10,
 '3564451': 11,
 '26661484': 12,
 '22476409': 13,
 '13374855': 14,
 '34873569': 15,
 '32064204': 16,
 '26622362': 17,
 '4117681': 18,
 '19917835': 19,
 '19360412': 20,
 '18316983': 21,
 '29054304': 22,
 '33791616': 23,
 '24565614': 24,
 '29154521': 25,
 '28029529': 26,
 '20077158': 27,
 '29071262': 28,
 '10570590': 29,
 '18729853': 30,
 '21699394': 31,
 '22133170': 32,
 '17718228': 33,
 '2780540': 34,
 '15468961': 35,
 '27561016': 36,
 '24388750': 37,
 '17699368': 38,
 '17516742': 39,
 '30144827': 40,
 '7803665': 41,
 '2757904': 42,
 '32786187': 43,
 '9138710': 44,
 '17968518': 45,
 '29167021': 46,
 '26176329': 47,
 '30204700': 48,
 '18295055': 49,
 '30619043': 50,
 '20048212': 51,
 '24722706': 52,
 '12945028': 53,
 '21263237': 54,
 '17075280': 55,
 '25390664': 56,
 '34800134': 57,
 '31247519': 58,
 '23537790': 5

In [ ]:
for index, row in new_df.iterrows():
    value = row['filtered_relevant_articles']
    #value = ast.literal_eval(value)
    new_list = []
    for paper in value:
        new_list.append(dict_of_articles[paper])
    new_df.at[index, 'filtered_relevant_articles'] = new_list
new_df.to_csv('extended_patient_relevant_articles_ids_map_with_full_text.csv', index=False)

new_df.head()

KeyError: 23531

In [85]:
new_df.head()

,PMID,filtered_relevant_articles,extended_extracted_symptoms
0,33492400,"[32320506, 32293716, 23219649, 30339549, 17470...","['acute ward', 'air distribution', 'ards', 'br..."
1,33492400,"[32320506, 32293716, 23219649, 30339549, 17470...","['aggravated', 'alleviate', 'anxiety', 'anxiou..."
2,33492400,"[32320506, 32293716, 23219649, 30339549, 17470...","['admitted', 'breathing frequency', 'convalesc..."
3,33492400,"[32320506, 32293716, 23219649, 30339549, 17470...","['active participation', 'admitted', 'ards', '..."
4,33492400,"[32320506, 32293716, 23219649, 30339549, 17470...","['abdomen', 'admitted', 'airway', 'alveolar co..."
